In [ ]:
# chrome webdriver
# request the following Uris
# api.clubelo.com/YYYY-MM-DD

In [ ]:
from selenium import webdriver
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd


Scrape data

In [ ]:
#get all HFA from 22 Jul 1993 - 2022 (both included)
driver = webdriver.Chrome()
# points = []


temp_list=[]

for country in ["ENG","GER","ESP","ITA"]:
    for year in range (1993, 2023):
        date = f"{year}-07-22"
        key = f"{date}/{country}"
        # key = league
        url = f'http://clubelo.com/{key}'
        driver.get(url)
        sleep(0.5)
        try:
            # paragraph = WebDriverWait(driver,30).until(EC.presence_of_element_located(driver.find_element(By.XPATH,'/html/body/div/div[3]/p/text()[1]'))
            paragraph = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,'/html/body/div/div[3]/p')))
            # paragraph = driver.find_element(By.XPATH,'/html/body/div/div[3]/p')
            # print(paragraph)
            paragraph_val = paragraph.text
            #split 0: remove new line
            paragraph_val = paragraph_val.split("\n")[0]
            #split 1: remove "Home Field Advantage: "
            paragraph_val = str(paragraph_val.split("Home Field Advantage: ")[1])
            #split 2: remove " Elo points."
            paragraph_val = float(paragraph_val.split(" Elo points.")[0])
        finally:
            driver.back()
        # points.append(paragraph_val)

        # save object in a list
        temp = {
            "DATE": date,
            "COUNTRY": country,
            "YEAR": year,
            "HFA": float(paragraph_val),
        }
        print(f"{temp} is being saved")
        temp_list.append(temp) # values to be used in dataframe that will be exported as a csv at the end
        sleep(61)

driver.quit()


Order data

In [ ]:
date = []
country = []
year = []
hfa = []

for temp in temp_list:
    date.append(temp["DATE"])
    country.append(temp["COUNTRY"])
    year.append(temp["YEAR"])
    hfa.append(temp["HFA"])

dataframe = pd.DataFrame({
    'DATE': date,
    'COUNTRY': country,
    'YEAR': year,
    'HFA': hfa})
print("Dates fetched")
print(dataframe)

print("saving dataframe")
dataframe.to_csv('../HFA/HFA_ENG_ESP_GER_ITA_1993_to_2022.csv')

Split HFA data obtained into separate files

In [ ]:
import pandas as pd

df = pd.read_csv("HFA_ENG_ESP_GER_ITA_1993_to_2022.csv")
df= df.drop(df.columns[0],axis=1)
print(df)
# dict = {country : pd.DataFrame()["DATE","COUNTRY","YEAR","HFA"] for country in ['ENG','ESP','GER','ITA']}

for country in ['ENG', 'ESP', 'GER', 'ITA']:

    df2 = df[:][df.COUNTRY == country]
    print(df2)
    # print(df_save.info())
    df2.to_csv(f'..\\{country}.csv',index=False)

In [ ]:
# chrome webdriver
# request the following Uris
# api.clubelo.com/YYYY-MM-DD
# MM = 08 (AUG) DD = 01
# YYYY varied'
import pandas as pd
import csv
import os
countries = ["ENG","GER","ESP","ITA"]
levels = ["1","2"]

for year in range (1993, 2023):
    file_path = f'./all_leagues/{year}-07-22.csv'
    df = pd.read_csv(file_path)
    # print(df)
    df = df.filter(["Rank","Club","Country","Level","Elo"])
    for country in countries:
        for level in levels:
            tmp = df.query(f"Country == '{country}' and Level == {level}")
            print(f"{year}: {len(df)}")
            wanted_path = f"./{country}/{level}"
            if not os.path.exists(wanted_path):
                os.makedirs(f"{wanted_path}/")
                tmp.to_csv(f"{wanted_path}/{year}-07-22.csv")
            else:
                tmp.to_csv(f"{wanted_path}/{year}-07-22.csv")